In [ ]:
import pandas as pd
import numpy as np

from sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import MultiLabelBinarizer, FunctionTransformer, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [ ]:
df = pd.read_csv("all_data.csv")

y = df["sold_price"]
X = df.drop("sold_price")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

cat_feats = df.dtypes[df.dtypes == 'object'].index.tolist()
num_feats = df.dtypes[~df.dtypes.index.isin(cat_feats)].index.tolist()
list_feats = []
for category in cat_feats:
    if type(df.loc[0, category]) == list:
        cat_feats.remove(category)
        list_feats.append(category)

In [ ]:
def numFeat(data):
    return data[num_feats]

def catFeat(data):
    return data[cat_feats]

def listFeat(data):
    return data[list_feats]

keep_num = FunctionTransformer(numFeat)
keep_cat = FunctionTransformer(catFeat)
keep_list = FunctionTransformer(listFeat)

In [ ]:
mlb = MultiLabelBinarizer()
minmax_scaler = MinMaxScaler()
imputer = SimpleImputer()
class_encoder = OneHotEncoder()

In [ ]:
# Potential models
base_model = Ridge()
rf_model = RandomForestRegressor()
gb_model = GradientBoostingRegressor()

In [ ]:
num_pipe = Pipeline([
    ('NumFilter', keep_num),
    ('MMScaler', minmax_scaler),
    ('Imputer', imputer)
    
])

cat_pipe = Pipeline([
    ('CatFilter', keep_cat),
    ('OHEncode', class_encoder),
    ('Imputer', imputer)
])

list_pipe = Pipeline([
    ('ListFilter', keep_list),
    ('MultiLabelBinarizer', mlb),
    ('Imputer', imputer)
])

pre_processing_pipeline = FeatureUnion([("num", num_pipe), ("cat", cat_pipe), ("list", list_pipe)])

In [ ]:
pipeline = Pipeline([("preprocessing", pre_processing_pipeline), ("model", None)])

In [ ]:
params = {
    "model": [base_model, rf_model, gb_model]
}

grid_search = GridSearchCV(pipeline, params, verbose=10, refit=True)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

In [ ]:
# save your best model here

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [ ]:
# Build pipeline here

Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [ ]:
# save your pipeline here